# Project: Blocks World (A*)
### by Samuel Sovi

A project designed to test whether or not a functional environment is created where blocks are able to stack, be stacked upon that can stack blocks in order in the fastest possible way.



##### Imports:
random to get random integers

In [ ]:
import random

BlockStack class which is implemented using a python list. The point of this class is to ensure that I am only adding to and removing from the top of a given BlockStack. The BlockStack class now has functionality for hashing itself and scoring the stack in terms of distance from ideal

In [ ]:
class BlockStack:
  def __init__(self, max_size):
    self.max = max_size
    self.stack = []
    pass
  
  #pushes a number block onto the stack
  def push(self, num):
    self.stack.append(num)
  
  #pops a number block off of the stack
  def pop(self):
    return self.stack.pop()
  
  #prints the stack
  def print_stack(self):
    print(self.stack)

  #returns whether or not stack is empty
  def is_empty(self):
    return len(self.stack) == 0
  
  #returns if the stack is in order and complete (max size)
  def is_full_order(self):
    if len(self.stack) < self.max:
      return False

    ordered = True
    for i in range(self.max - 1):
      if self.stack[i] > self.stack[i + 1]:
        ordered = False
    return ordered
  
  #hashes the stack in a sense
  def hash_stack(self):
    hash = ""
    for i in range(len(self.stack)):
      hash = hash + str(self.stack[i]) + "."
    hash = hash + "|"
    return hash
  
  #scores the stack based on distance from "ideal" (lowest score is best)
  def score_stack(self, target):
    found = False # found target
    order = False # stack on top of 1 is ordered correctly without skips
    order2 = False # stack in descending order (not on 1)
    score = 0

    for i in range(len(self.stack)):
      if self.stack[i] == target:
        found = True
        if target == 1 and i == 0:
          order = True
          score = score - 10
          #decrease score if 1 is at the bottom
      elif found == True:
        if self.stack[i] != self.stack[i-1] + 1:
          order = False
          # update not ordered if not proper stack on top of 1
        if order == True:
          score = score - 10
          #decrease score if proper stack on top of 1
        else:
          score = score + 5
          #increase score if unordered stack
      else:
        if i == 0:
          order2 = True
          #set order 2 = True if 1 is not bottom of stack
        elif i > 0 and self.stack[i] > self.stack[i-1]:
          score = score + 5
          # add score if not reverse order
          order2 = False
        elif order2 == False:
          score = score + 5
          # add score if reverse order was already broken before
    return score
  
  # returns the list of elements (for copying purposes)
  def get_vals(self):
    return self.stack
    

        
    


  


BlocksWorld class is a list of BlockStacks which creates the initial environment of blocks and moves them in order for solving. The basic gist of the algorithm is that it prioritizes moving 1 to the bottom of a stack. Then priotize stacking correctly on top of 1. Then prioritize breaking stacks that are in ascending order (and not on top of 1). 

In [ ]:
class BlocksWorld:
  def __init__(self, size):
    self.world = []
    self.calc_counter = 0
    #self.hash_list = []
    for i in range(size):
      self.world.append(BlockStack(size))
    
    #note: not truly random since 1 is guaranteed to be at bottom, etc:
    numbers = []
    for i in range(size):
      numbers.append(i+1)
    random.shuffle(numbers)
    for i in range(size):
      self.world[random.randint(0,size-1)].push(numbers[i])
  
  #prints out the world
  def print_world(self):
    for a in range(len(self.world)):
      self.world[a].print_stack()
  
  #returns whether or not the world is complete
  def is_complete(self):
    for i in range(len(self.world)):
      if self.world[i].is_full_order():
        return True
    return False
  
  #random solves (with single steps)
  def solve(self):
    counter = 0
    #self.hash_list.append(self.self_hash(counter))
    stuck = False;
    while not self.is_complete() and not stuck:
      counter = counter + 1
      stuck = self.move_block(counter)
      # if block number > 4 then only display every millionth move after first 20
      if(counter <= 40):
        print("Move Counter =", counter)
        print("Calculated States =", self.calc_counter)
        #print("Score =", self.self_score(1))
        self.print_world()
        #print(self.hash_list[-1])
        #print("yo", len(self.hash_list))
        print("\n")
    
    if stuck:
      #uses stuck check if using the move_block that supports "no repeat moves"
      print("Stuck due to 'no repeated state' constraint after", counter, "moves")
      self.print_world()
      return

    print("Solving Complete")
    print("Moves Taken:", counter)
    print("Total Calculated States =", self.calc_counter)
    print("World: \n")
    self.print_world()
  
  

  
  # recursion version (without checking for repeats)
  # this makes sure that the run does not get stuck 
  # (and doesn't take forever due to previous state checks)
  def move_block(self, steps):
    min_score = self.self_score(1)
    min_world = [None]*len(self.world)
    # for every stack in the world
    for i in range(len(self.world)):
      # if the stack is not empty
      if not self.world[i].is_empty():
        # for every stack in the world
        for a in range(len(self.world)):
          if a != i:
            # if the two stacks are not the same, move block 
            # from 1 stack to the other, then calculate the score
            # for the new world
            # if the score is lower than the current lowest, keep it
            self.world[a].push(self.world[i].pop())
            self.calc_counter = self.calc_counter + 1
            if self.self_score(1) < min_score:
              min_score = self.self_score(1)
              # copy world or get hash
              for b in range(len(self.world)):
                min_world[b] = BlockStack(len(self.world))
                num_list = self.world[b].get_vals()
                for c in range(len(num_list)):
                  min_world[b].push(num_list[c])

            self.world[i].push(self.world[a].pop())
    # copy world of best score
    for b in range(len(min_world)):
      self.world[b] = BlockStack(len(min_world))
      num_list = min_world[b].get_vals()
      for c in range(len(num_list)):
        self.world[b].push(num_list[c])
    return False
    
  # hashing function for keeping track of previous states
  def self_hash(self):
    hash = ""
    for i in range(len(self.world)):
      hash = hash + self.world[i].hash_stack()
    hash = hash
    return hash
  
  
  def self_score(self, target):
    score = 0
    for i in range(len(self.world)):
      score = score + self.world[i].score_stack(target)
    return score
  
  #def world_from_hash(self, hash):
    




The following code cell contains my creation of a 7 block blocksworld. 

In [ ]:
small_world = BlocksWorld(7)
small_world.print_world()

[7]
[]
[2, 4, 3]
[1, 5]
[]
[6]
[]


The following code cell contains the solving of a 7 block blocksworld. The output contains the first 40 moves for demonstration purposes. Also printing each step (rather than every calculated step) as stated in class today.

In [ ]:
small_world.solve()

Move Counter = 1
Calculated States = 24
[7, 3]
[]
[2, 4]
[1, 5]
[]
[6]
[]


Move Counter = 2
Calculated States = 48
[7, 3]
[4]
[2]
[1, 5]
[]
[6]
[]


Move Counter = 3
Calculated States = 78
[7, 3]
[4]
[2]
[1]
[5]
[6]
[]


Move Counter = 4
Calculated States = 114
[7, 3]
[4]
[]
[1, 2]
[5]
[6]
[]


Move Counter = 5
Calculated States = 144
[7]
[4]
[]
[1, 2, 3]
[5]
[6]
[]


Move Counter = 6
Calculated States = 174
[7]
[]
[]
[1, 2, 3, 4]
[5]
[6]
[]


Move Counter = 7
Calculated States = 198
[7]
[]
[]
[1, 2, 3, 4, 5]
[]
[6]
[]


Move Counter = 8
Calculated States = 216
[7]
[]
[]
[1, 2, 3, 4, 5, 6]
[]
[]
[]


Move Counter = 9
Calculated States = 228
[]
[]
[]
[1, 2, 3, 4, 5, 6, 7]
[]
[]
[]


Solving Complete
Moves Taken: 9
Total Calculated States = 228
World: 

[]
[]
[]
[1, 2, 3, 4, 5, 6, 7]
[]
[]
[]


The following code cell contains my creation of a 9 block blocksworld.

In [ ]:
world = BlocksWorld(9)
world.print_world()

[1, 4]
[3]
[5, 8, 6]
[]
[]
[7]
[2, 9]
[]
[]


The following code cell contains the solving of a 9 block blocksworld. The output contains the first 40 moves for demonstration purposes. Also printing each step (rather than every calculated step) as stated in class today.

In [ ]:
world.solve()

Move Counter = 1
Calculated States = 40
[1]
[3]
[5, 8, 6]
[4]
[]
[7]
[2, 9]
[]
[]


Move Counter = 2
Calculated States = 88
[1]
[3]
[5, 8]
[4]
[6]
[7]
[2, 9]
[]
[]


Move Counter = 3
Calculated States = 144
[1]
[3]
[5]
[4]
[6]
[7]
[2, 9]
[8]
[]


Move Counter = 4
Calculated States = 208
[1]
[3]
[5]
[4]
[6]
[7]
[2]
[8]
[9]


Move Counter = 5
Calculated States = 280
[1, 2]
[3]
[5]
[4]
[6]
[7]
[]
[8]
[9]


Move Counter = 6
Calculated States = 344
[1, 2, 3]
[]
[5]
[4]
[6]
[7]
[]
[8]
[9]


Move Counter = 7
Calculated States = 400
[1, 2, 3, 4]
[]
[5]
[]
[6]
[7]
[]
[8]
[9]


Move Counter = 8
Calculated States = 448
[1, 2, 3, 4, 5]
[]
[]
[]
[6]
[7]
[]
[8]
[9]


Move Counter = 9
Calculated States = 488
[1, 2, 3, 4, 5, 6]
[]
[]
[]
[]
[7]
[]
[8]
[9]


Move Counter = 10
Calculated States = 520
[1, 2, 3, 4, 5, 6, 7]
[]
[]
[]
[]
[]
[]
[8]
[9]


Move Counter = 11
Calculated States = 544
[1, 2, 3, 4, 5, 6, 7, 8]
[]
[]
[]
[]
[]
[]
[]
[9]


Move Counter = 12
Calculated States = 560
[1, 2, 3, 4, 5, 6, 7,